<a href="https://colab.research.google.com/github/jahnvisikligar/QMUL-Datamining/blob/master/Lab05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab session 5: Classification and clustering

## Introduction

The purpose of this lab session is to provide you with an opportunity to gain experience in **classification** and **clustering**, covered in week 6, using common Python libraries.

- This lab is the second part of a **two-week assignment** that covers weeks 5 and 6, which is due on **Friday 12th November 10am**.
- The assignment will account for 10% of your overall grade. Questions in this lab sheet will contribute to 5% of your overall grade; questions in the lab sheet for week 5 will cover for another 5% of your overall grade.
- <font color = 'maroon'>The last section of this notebook includes the questions that are assessed towards your final grade.</font> 

This session starts with a tutorial that uses examples to introduce you to the practical knowledge that you will need for the corresponding assignment. We highly recommend that you read the following tutorials if you need a gentler introduction to the libraries that we use:
- [Numpy quickstart tutorial](https://numpy.org/devdocs/user/quickstart.html)
- [Numpy: basic broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html)
- [Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html)
- [Matplotlib](https://matplotlib.org/tutorials/introductory/pyplot.html)
- [Seaborn](https://seaborn.pydata.org/tutorial/relational.html)
- [Scikit-learn](https://scikit-learn.org/stable/tutorial/basic/tutorial.html)

## Important notes about the assignment: 

- **PLAGIARISM** <ins>is an irreversible non-negotiable failure in the course</ins> (if in doubt of what constitutes plagiarism, ask!). 
- The total assessed coursework is worth 40% of your final grade.
- There will be 9 lab sessions and 4 assignments.
- One assignment will cover 2 consecutive lab sessions and will be worth 10 marks (percentages of your final grade).
- The submission cut-off date will be 7 days after the deadline and penalties will be applied for late submissions in accordance with the School policy on late submissions.
- You are asked to submit a **report** that should answer the questions specified in the last section of this notebook. The report should be in a **single PDF document** (so **NOT** *doc, docx, notebook* etc). This single PDF document will include your answers to both the week 5 and week 6 labs.
- No other means of submission other than submitting your assignment through the appropriate QM+ link are acceptable at any time. Submissions sent via email will **not** be considered.
- Please name your report as follows: Assignment2-StudentName-StudentNumber.pdf
- Cases of **Extenuating Circumstances (ECs)** have to go through the proper procedure of the School in due time. Only cases approved by the School in due time can be considered.

## Classification

In order to present functionalities for data classification and clustering in Python, we use the [MNIST handwritten digits dataset](http://yann.lecun.com/exdb/mnist/) as part of a working example.

The dataset is stored in a file called ``mnist.pkl.gz``, which is compressed by [gzip](https://docs.python.org/3/library/gzip.html) and serialized by [pickle](https://docs.python.org/3/library/pickle.html). 

In the snippet below, ``X`` is a matrix (numpy array) where each row corresponds to an observation and each column corresponds to a feature. Each observation is the result of *flattening* a 28 x 28 grayscale image of a handwritten digit into a vector. The list ``y`` (numpy array) contains the class (a digit from 0 to 9) corresponding to each observation (row) in the matrix ``X``.

For the sake of this tutorial, we select only the first 2000 observations (rows) from the original MNIST training dataset to expedite computations.

It is possible to use the ``matplotlib`` function ``imshow`` to visualise any observation by reshaping it appropriately. You may change the index ``i`` to select an observation for visualisation.




In [ ]:
# Configuring the appearance of ``seaborn`` graphics in this notebook
%config InlineBackend.figure_formats = set(['retina'])

import gzip
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

# Selecting the training data from the original dataset
f = gzip.open('data/mnist.pkl.gz', 'rb')
X, y = pickle.load(f, encoding='latin1')[0]
f.close()

# Subsampling
sample_size = 2000
X, y = X[:sample_size], y[:sample_size]

i = 10
plt.imshow(X[i].reshape(28, 28), cmap=plt.cm.gray_r, interpolation='nearest')
plt.show()
print('Observation index: {0}. Class: {1}.'.format(i, y[i]))

FileNotFoundError: ignored

The library ``sklearn`` implements all the learning algorithms that we covered during the lectures. Each algorithm is implemented by a class that provides a standard interface.

The class ``KNeighborsClassifier`` implements a k-nearest neighbour classification algorithm. The number of neighbours is specified by the constructor parameter ``n_neighbors``.

The method ``KNeighborsClassifier.fit`` is responsible for learning a classifier for a dataset represented by an observation matrix and a class array. This is precisely how our data is represented.

The method ``KNeighborsClassifier.score`` computes the accuracy of a classifier on a specific dataset. This method must be called after ``KNeighborsClassifier.fit``.

As we have seen, a one-nearest neighbour classifier always predicts the correct class for any observation that already exists in a dataset (as long as there are no equal observations with different classes), which is why we obtain 100% accuracy on the training set.


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X, y)
print('Training dataset accuracy: {0}.'.format(knn.score(X, y)))

NameError: ignored

In order to evaluate the capacity of a k-nearest neighbour classifier to assign unseen observations to correct classes, it is necessary to split the original dataset into a training dataset and a test dataset.

The ``sklearn`` function ``train_test_split`` can be used for this purpose. This function selects a fraction of the observations (``test_size``) to compose the test set. The parameter ``random_state`` allows reproducibility by fixing the seed used by this (pseudo)random selection procedure. 

The function ``train_test_split`` returns a training observation matrix, a test observation matrix, a training class array, and a test class array. From now on, all classifiers are trained without access to the test dataset.

The method ``KNeighborsClassifier.fit`` can be used to fit a k-nearest neighbour classifier to the training dataset. Naturally, the accuracy of the one-nearest neighbour classifier on the training set remains 100%. The accuracy on the test set, however, is a better estimate of the performance of the classifier on unseen data.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
print('Training dataset accuracy: {0}.'.format(knn.score(X_train, y_train)))
print('Test dataset accuracy: {0}.'.format(knn.score(X_test, y_test)))

NameError: ignored

After a ``KNeighborsClassifier`` is fit, the method ``KNeighborsClassifier.predict`` can be used to predict the classes for a list of observations.

For example, we may use this method to predict the class for every observation in the test dataset. This allows obtaining the indices of the observations that are classified incorrectly.

In [ ]:
y_pred = knn.predict(X_test)
mistakes = np.nonzero(y_pred != y_test)[0]

print('Indices of misclassified observations:')
print(mistakes)

We can use the ``matplotlib`` function ``imshow`` to visualise a misclassified observation. You may change the index for ``mistakes`` in order to select an observation for visualisation (requires running all cells above, because ``y_pred`` is modified later). Note how some misclassifications are understandable. 

In [ ]:
i = mistakes[1]
plt.imshow(X_test[i].reshape(28, 28), cmap=plt.cm.gray_r, interpolation='nearest')
plt.show()
print('Observation index: {0}. Class: {1}. Prediction: {2}.'.format(i, y_test[i], y_pred[i]))

Some datasets need to be preprocessed before a learning algorithm can be applied. The training dataset must be preprocessed without access to the test dataset, otherwise the resulting test accuracy after preprocessing is likely to overestimate the accuracy on unseen data.

The function ``make_pipeline`` allows chaining a sequence of preprocessing operations (such as individually standardizing each feature, which is accomplished by the class ``StandardScaler``) before a learning algorithm is applied. This function returns a ``Pipeline`` object that can be used to fit, predict, and score, just as any other classification algorithm in ``sklearn``. A ``Pipeline`` preprocesses incoming data appropriately before making predictions.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

knn_pipe = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=1))

knn_pipe.fit(X_train, y_train)
print('Test dataset accuracy: {0}.'.format(knn_pipe.score(X_test, y_test)))

Consider the task of selecting the best possible number of neighbours k (a hyperparameter) for the k-nearest neighbour classification algorithm.

As we have seen, a hyperparameter should not be chosen based on its performance on the test dataset. In that case, there would be no data left to enable a reliable estimate of how well this choice generalizes to unseen data. After all, this choice would have been based on the test dataset.

The function ``cross_validate`` conducts K-fold cross validation using a specified training dataset. Recall that cross-validation involves splitting a training dataset into K folds of equal size. Each fold is used as a validation set (where accuracies are computed) when the remaining folds are used as an effective training set (where a classifier is learned). The parameter ``cv`` controls the number of folds K. The function returns a dictionary with statistics that include a list of accuracies that can be accessed by the keyword ``'test_score'``.

After a hyperparameter is selected by cross-validation, it can be used to fit a classifier to the entire training dataset, which can then be evaluated in the test dataset.

Once again, if any step leading to model selection is based on the resulting performance on the test dataset, the resulting test accuracy is likely to overestimate the accuracy of the model on unseen data.

In [ ]:
knn_5 = KNeighborsClassifier(n_neighbors=5)
knn_15 = KNeighborsClassifier(n_neighbors=15)

from sklearn.model_selection import cross_validate

# Note that the data was already implicitly shuffled by train_test_split
result_5 = cross_validate(knn_5, X_train, y_train, cv=5)
result_15 = cross_validate(knn_15, X_train, y_train, cv=5)

print('Average accuracy across folds (k = 5): {0}.'.format(result_5['test_score'].mean()))
print('Average accuracy across folds (k = 15): {0}.'.format(result_15['test_score'].mean()))
      
knn_5.fit(X_train, y_train)
print('Test dataset accuracy (k = 5): {0}.'.format(knn_5.score(X_test, y_test)))

The class ``GridSearchCV`` offers a convenient way to choose hyperparameters based on cross-validation. Its constructor receives a classification algorithm object (such as a ``KNeighborsClassifier`` object) and a dictionary that maps hyperparameter names to lists of values that should be considered. Once the method ``GridSearchCV.fit`` is called, each possible combination of hyperparameters from each of the lists is evaluated using cross-validation. The best hyperparameter setting is then used to fit a classifier to the training dataset. After fitting, the ``GridSearchCV`` object can be used to predict and score just as any other classification algorithm in ``sklearn``.

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'n_neighbors': [1, 3, 5, 7, 9]}

knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, parameters, cv=5)
knn_cv.fit(X_train, y_train)

print('Best hyperparameter setting: {0}.'.format(knn_cv.best_estimator_))
print('Average accuracy across folds of best hyperparameter setting: {0}.'.format(knn_cv.best_score_))
print('Test dataset accuracy of best hyperparameter setting: {0}.'.format(knn_cv.score(X_test, y_test)))

Accuracy is not the only classification performance metric available on ``sklearn``. For instance, it is possible to compute the precision and recall for each class by considering the remaining classes as beloging to a single (negative) class. This is accomplished by the function ``precision_recall_fscore_support``.

The function ``confusion_matrix`` creates a confusion matrix given a class array and a prediction array.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

y_pred = knn_cv.predict(X_test)

precision, recall, _, _ = precision_recall_fscore_support(y_test, y_pred)
print('Precision for each class: {0}.'.format(precision))
print('Recall for each class: {0}.\n'.format(recall))

df = pd.DataFrame.from_records(confusion_matrix(y_test, y_pred))
print('Confusion matrix:')
display(df)

As mentioned before, ``sklearn`` implements learning algorithms covered during the lectures:  [support vector machines](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html), [artificial neural networks](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html), [decision trees](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html), and [random forest classifiers](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

The interface provided by each of these learning algorithms is analogous to the interface provided by the k-nearest neighbour classification algorithm that we have used so far.

For instance, in order to train a random forest classifier composed of 100 decision trees, an object of the class ``RandomForestClassifier`` can be constructed with ``n_estimators`` set to 100. This object can be used to fit the training dataset and compute the accuracy on the test dataset.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100, random_state=0)
rfc.fit(X_train, y_train)
print(rfc.score(X_test, y_test))
print('Test dataset accuracy (random forest classifier): {0}.'.format(rfc.score(X_test, y_test)))

## Clustering

The clustering algorithms implemented by ``sklearn`` have an interface that is similar to the interface for the classification algorithms.

The class ``KMeans`` implements the k-means clustering algorithm. The number of clusters ``n_clusters`` is a parameter for the constructor of this class. 

The method ``KMeans.fit_predict`` is equivalent to a call to ``KMeans.fit`` followed by a call to ``KMeans.predict``, which is responsible for attributing each observation to a cluster. After the clustering is computed by a call to ``KMeans.fit``, the sum of squared errors of the clustering is available as a class variable named ``inertia_``.


In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, random_state=0)
y_pred = kmeans.fit_predict(X)

print('Sum of squared errors (k = 10): {0}.'.format(kmeans.inertia_))

In order to compare the quality of clusterings of the same dataset for different numbers of clusters, it is possible to employ both sums of squared errors and silhouette coefficients.

In [ ]:
from sklearn.metrics import silhouette_score

max_k = 15

# Sum of squared errors for each k
sses = []

# Silhouette coefficient for each k
silhouettes = []

for k in range(2, max_k + 1):
    kmeans = KMeans(n_clusters=k, random_state=0)
    y_pred = kmeans.fit_predict(X)
    
    sses.append(kmeans.inertia_)
    silhouettes.append(silhouette_score(X, y_pred))

# Plotting sums of squared errors
df = pd.DataFrame({'sum of squared errors': sses, 'number of clusters': list(range(2, max_k + 1))})
sns.lineplot(x='number of clusters', y='sum of squared errors', data=df)
plt.xticks(df['number of clusters'])
plt.show()

# Plotting silhouette coefficients
df = pd.DataFrame({'silhouette coefficient': silhouettes, 'number of clusters': list(range(2, max_k + 1))})
sns.lineplot(x='number of clusters', y='silhouette coefficient', data=df)
plt.xticks(df['number of clusters'])
plt.show()

NameError: ignored

A projection computed using t-stochastic neighbor embedding (implemented by the class ``TSNE``) can be used to visualise the results of clustering. 

Recall that dimensionality reduction attemps to represent a dataset by a projection such that each point in the projection corresponds to an observation. The t-stochastic neighbor embedding algorithm attempts to preserve neighbourhoods of the dataset in the corresponding projection.

It is important to note that a projection is not, in general, a reliable representation of a dataset. Therefore, care must be taken before deriving conclusions from such visualisations. 

Note that the numbers assigned to clusters are unlikely to match the numbers assigned to classes. Therefore, only relative colouring is important in the plots shown below.

In [ ]:
from sklearn.manifold import TSNE

# Computing a projection using t-stochastic neighbour embedding
embedding = TSNE(n_components=2, perplexity=50, random_state=0)
Xp = embedding.fit_transform(X)

# Plotting projection colored by classes
df_projection = pd.DataFrame({'x': Xp[:, 0], 'y': Xp[:, 1], 'class': y})
sns.scatterplot(x='x', y='y', hue='class', palette=sns.color_palette(), data=df_projection)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

kmeans = KMeans(n_clusters=10, random_state=0)
y_pred = kmeans.fit_predict(X)

# Plotting projection colored by clusters
df_projection = pd.DataFrame({'x': Xp[:, 0], 'y': Xp[:, 1], 'cluster': y_pred})
sns.scatterplot(x='x', y='y', hue='cluster', palette=sns.color_palette(), data=df_projection)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

NameError: ignored

## Assignment 2 [part 2 of 2]

For your answers to the assignment, please include include your workings (e.g. equations, code) when this is relevant to the question. Questions 1-5 are pen-and-paper exercises. Questions 6 and 7 below require you to write and provide code.

1. Consider a dataset $\mathcal{D}$ that contains only two observations $\mathbf{x}_{1} = (1, -1)$ and $\mathbf{x}_2 = (-1, 1)$. Suppose that the class of the first observation is $y_1 = 0$ and that the class of the second observation is $y_2 = 1$. How would a 1-nearest neighbour classifier based on the Euclidean distance classify the observation $\mathbf{x} = (-2, 3)$? What are the distances between this new observation and each observation in the dataset? [0.5 marks out of 5]
2. Consider a dataset $\mathcal{D}$ that only contains observations of two different classes. Explain why a $k$-nearest neighbour classifier does not need a tie-breaking policy when $k$ is odd. [0.5 marks out of 5]
3. Explain why a classifier that obtains an accuracy of $99.9\%$ can be terrible for some datasets. [0.5 marks out of 5]
4. Consider a classifier tasked with predicting whether an observation belongs to class $y$ (positive class). Suppose that this classifier has precision $1.0$ and recall $0.1$ on a test dataset. If this classifier predicts that an observation does not belong to class $y$, should it be trusted? Should it be trusted if it predicts that the observation belongs to class $y$? [0.5 marks out of 5]
5. Based on the confusion matrix shown in this lab notebook, what is the pair of classes that is most confusing for the $1$-nearest neighbour classifier trained in the previous sections? [0.5 marks out of 5]
6. Train a support vector machine classifier using the same (subsampled) training dataset used in the previous sections and compute its accuracy on the corresponding test dataset. You can use the default hyperparameters for the class ``SVC`` from ``sklearn.svm``. Show the code in the report. [1.25 marks out of 5]
7. Using the same (subsampled) training dataset used in the previous sections, employ ``GridSearchCV`` to find the best hyperparameter settings based on 5-fold cross-validation for a ``RandomForestClassifier``. Consider ``n_estimators`` $ \in \{ 50, 100, 200\}$ and ``max_features`` $ \in \{0.1, 0.25\}$. Use the default values for the remaining hyperparameters. Compute the accuracy of the best model on the corresponding test dataset. Show the code in the report. [1.25 marks out of 5]

In [ ]:
def kmeans_update(X, cluster_centers):
 y_pred = np.argmin(cdist(X, cluster_centers), axis=1)

 next_cluster_centers = np.zeros(cluster_centers.shape)
 for i in range(len(next_cluster_centers)):
     next_cluster_centers[i] = X[y_pred == i].mean(axis=0)

 return y_pred, next_cluster_centers